# Nasdaq Machine Learning

## Stock price data maipulation, visualisation and machine learning on Nasdaq listed companies.

First import all the relevant modules required. Set up the program.

In [1]:
import bs4 as bs
import datetime as dt
import os
import pandas_datareader.data as web
import pandas as pd
import pickle
import requests
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
from collections import Counter
from sklearn import svm, model_selection, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

style.use('ggplot')

ModuleNotFoundError: No module named 'pandas_datareader'

We begin by collecting the tickers for all the Nasdaq listed companies.

vist Wiki page, get response which has the source code
turn the .text attribute to soup using BeautifulSoup
(BS turns source code to a BS object that you can treat more like Python object)

the specific solution to searching through the wiki table on that specific page
had to look into the sourcecode
we find in html the table has class="wikitable sortable"

for each row after header row, ticker is table data td, grab .text of it and append to tickers list

save the tickers list with pickle

In [ ]:
def save_nasdaq_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/NASDAQ-100')
    soup = bs.BeautifulSoup(resp.text, 'lxml')

    table = soup.find('table', {'class': 'wikitable sortable'})
   
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text[:-1]
        tickers.append(ticker)

    with open("nasdaqtickers.pickle","wb") as f:
        pickle.dump(tickers,f)

    print(tickers)

    return tickers

save_nasdaq_tickers()

Next, collect the data from the Nasdaq list from Yahoo Finance.

handle for whether or not to reload the nasdaq list
if we ask it to, program will re-pull nasdaq list
else, use our pickle

we want to pull all the data from Yahoo for every sock and save it
first, create new directory with stock data per company

pull the data

In [ ]:
def get_nasdaq_data_from_yahoo(reload_nasdaq=False):
    if reload_nasdaq:
        tickers = save_nasdaq_tickers()
    else:
        with open("nasdaqtickers.pickle", "rb") as f:
            tickers = pickle.load(f)

    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2017, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        print(ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))
            
get_nasdaq_data_from_yahoo()

Compile the data with Pandas.
join stock datasets together

pull previously made list of tickers, begin with an empty dataframe called main_df

read in each stock's dataframe

we are only interested in the Adj Close data

if there is nothing in main_df,  then start with current df, otherwise, use the Pandas join function

output the count of the current ticker if it's evenly divisible by 10

In [ ]:
def compile_data():
    with open("nasdaqtickers.pickle","rb") as f:
        tickers = pickle.load(f)
    main_df = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        if count % 10 == 0:
            print(count)

    print(main_df.head())
    main_df.to_csv('nasdaq_joined_closes.csv')
    
comile_data()

Form a heatmap of the data.

build a correlation table from the nasdaq_joined_closes data, save into csv
 
make a heatmap:

first, need actual data itself to graph
get numpy array of just the values (which are correlation numbers)

build the figure and axis

create the heatmap with pcolor
use the RdYlGn colormap - gives red for negative correlations, green for positive correlations,
yellow for no-correlations

set x and y axis so we know which companies are which
create tick markers

flip yaxis so that graph is easier to read (as there is more space between x's and y's
flip xaxis to be at top of graph (to make it more like a correlation table)

add company names to currently nameless ticks:

rotate xtics, which are the actual tickers themselves, as they would normally be written out horizontally
tell color map that range is from -1 to +1

In [ ]:
def visualize_data():
    df = pd.read_csv('nasdaq_joined_closes.csv')

    df_corr = df.corr()
    print(df_corr.head())
    df_corr.to_csv('nasdaq_corr.csv')

    data1 = df_corr.values

    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)

    heatmap1 = ax1.pcolor(data1, cmap=plt.cm.RdYlGn)
    fig1.colorbar(heatmap1)

    ax1.set_xticks(np.arange(data1.shape[1]) + 0.5, minor=False)
    ax1.set_yticks(np.arange(data1.shape[0]) + 0.5, minor=False)

    ax1.invert_yaxis()
    ax1.xaxis.tick_top()

    column_labels = df_corr.columns
    row_labels = df_corr.index
    ax1.set_xticklabels(column_labels)
    ax1.set_yticklabels(row_labels)

    plt.xticks(rotation=90)
    heatmap1.set_clim(-1,1)
    plt.tight_layout()
    # plt.savefig("correlations.png", dpi = (300))
    plt.show()

visualise_data()

Process the data to create labels.
function takes one ticker as parameter

each model is trained on a single company
next we need to know number of days into the future we need prices for (7 days)
read in data for close prices for all companies saved in past
get list of existing tickers
fill any missing with 0 for now
now we want to grab the % changed values for next 7 days

create dataframe columns for specific ticker using string formatting to create custom names
get future values with .shift which shifts a column up or down
shift a negative amount which will take that column and would shift column up by i rows
gives future values i days in advanced which can calculate percentage change against

return tickers and dataframe
now we can have some feature sets that the algorithms can use to find relationships

In [ ]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('nasdaq_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
  
    for i in range(1, hm_days + 1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
   
    df.fillna(0, inplace=True)
    return tickers, df

process_data_for_labels()

The buy, sell hold funciton.
if the price rises more than 2% in the next 7 days - this is a buy
if it drops more than 2% in next 7 days - this is a sell
if doesn't do either or these - this is a hold
this will be mapped to a Pandas DataFrame column

In [ ]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

Extract Featuresets.

take any ticker, create the dataset, create the target column (the label)
target column will have wither -1, 0 or  for each row
get the distribution
clean up data
some data may be missing - replace with 0
some data may be infinite - convert to NaN, and drop the NaN
features metric should be the company's percent change that day instead of the day's price stocks
(companies will change in price before others)
convert stock prices to % changes

create features and labels

X contains featuresets (daily & changes for every Nasdaq company)
y is the target (the label) - to map the featuresets to

In [ ]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)

    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)] ))

    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X = df_vals.values
    y = df['{}_target'.format(ticker)].values

    return X,y,df

Machine Learning with K-Nearest Neighours

shuffle data, create training and testing samples
apply K Nearest Neighbors classifer
train classifier on data
take X data and fit to the y data, for each pairs of X's an y's
take some featuresets, X_test, make a prediction, see if it matches labels, y_test
return percentage accuracy in decimal form
print accuracy
get predictions of X_testdata, output distribution using counter

In [ ]:
def machine_learning_KNNC(ticker):
    X, y, df = extract_featuresets(ticker)

    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

    clf = neighbors.KNeighborsClassifier()

    clf.fit(X_train, y_train)
    
    confidence = clf.score(X_test, y_test)

    print('~ results with K nearest neighbours classifier for', ticker, '~')
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    print()
    print()

Machine Learning with voting classifier. Procedure is similar to the method above.

In [ ]:
def machine_learning_VC(ticker):
    X, y, df = extract_featuresets(ticker)
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)
    clf = VotingClassifier([('lsvc', svm.LinearSVC()),
                            ('knn', neighbors.KNeighborsClassifier()),
                            ('rfor', RandomForestClassifier())])
    
    clf.fit(X_train, y_train)
    
    confidence = clf.score(X_test, y_test)
    
    print('~ results with voting classifier for', ticker, '~')
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    print()
    print()

In [ ]:
Now we can apply the machine learning methods to the company GOOGL.

In [ ]:
machine_learning_KNNC('GOOGL')
machine_learning_VC('GOOGL')